<a href="https://colab.research.google.com/github/sophiamaria05/IC_MDA/blob/main/xml_to_plantuml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing

In [ ]:
#@title Defines break and tab for writing the puml file
_break = """
"""
_tab = '  '

##Importing libraries

In [ ]:
#reading xml files
import xml.etree.cElementTree as ET
#importing files
import os

##Defining methods

In [ ]:
#@title ***get_xml_files(xml_file_folder):*** Geting the xml files (of the phrases)
def get_xml_files(xml_file_folder):
  return [file for file in os.listdir(xml_file_folder) if file.endswith('.xml')]

In [ ]:
#@title ***tag_to_array(tag):*** Converts phrase's ***type*** to array to make checking each part of the tag easier
def tag_to_array(tag):
  tag_array = [int(tag_part) for tag_part in tag.split(".")]
  while len(tag_array) < 3:
    tag_array.append(0)
  return tag_array

In [ ]:
#@title ***get_classes_and_methods(root_xml):*** Creates a dictionary with the classes and it's methods
def get_classes_and_methods(root_xml):
  classes_and_methods = {}
  associations = {}#                                                                                              ASSOCIATION TEST *****
  for use_case in root_xml:
    for flow in use_case:
      inputs = []
      for phrase in flow:
        nodes = [node.text for node in phrase.iter()]
        tag = tag_to_array(phrase.get("type"))
        if tag[0]==0:
          inputs = []
          continue
        if tag[2]==1:#special case (informa and insere)
          if (phrase.find("verb").text=="Informar" or phrase.find("verb").text=="Informa") and tag[1]==1:
            nodes = [node.text for node in phrase.iter()]
            nodes.pop(0)#pops the phrase.text
            use_case_name = "_".join(use_case.text.split(" "))
            method_name = "Informa(msg)"
            try:
              classes_and_methods["V_"+use_case_name+"  <<view>>"].append(method_name)
            except:
              classes_and_methods["V_"+use_case_name+"  <<view>>"]=[method_name]

          elif (phrase.find("verb").text=="Inserir" or phrase.find("verb").text=="Insere"):# and tag[1]==1:#      INSERE TEST *****
            nodes = [node.text for node in phrase.iter()]
            nodes.pop(0)#pops the phrase.text
            inputs = nodes[2:]

            attributes = inputs
            class_name = attributes.pop()
            for attribute_name in attributes:
              try:
                classes_and_methods[class_name].append(attribute_name)
              except:
                classes_and_methods[class_name]=[attribute_name]

        elif tag[1]==0:
          use_case_name = "_".join(use_case.text.split(" "))
          nodes = [node for node in phrase.iter()]
          class_name = nodes[-1].text
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))

          if inputs!=[]:#                                                                                         INSERE TEST *****
            method_name+=("("+",".join(inputs))+")"
            inputs = []
          else:
            method_name+="()"

          try:
            classes_and_methods["V_"+use_case_name+"  <<view>>"].append(method_name)
          except:
            classes_and_methods["V_"+use_case_name+"  <<view>>"]=[method_name]
          try:
            classes_and_methods["C_"+use_case_name+"  <<controller>>"].append(method_name)
          except:
            classes_and_methods["C_"+use_case_name+"  <<controller>>"]=[method_name]

          try:#                                                                                                 ASSOCIATION TEST *****
            associations["C_"+use_case_name].append("V_"+use_case_name)
          except:
            associations["C_"+use_case_name]=["V_"+use_case_name]
          if not (class_name in associations["C_"+use_case_name]):
            associations["C_"+use_case_name].append(class_name)

        else:
          nodes = [node for node in phrase.iter()]
          class_name = nodes[-1].text
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))

          if inputs!=[]:#                                                                                       INSERE TEST *****
            method_name+=("("+",".join(inputs))+")"
            inputs = []
          else:
            method_name+="()"

          try:
            classes_and_methods[class_name].append(method_name)
          except:
            classes_and_methods[class_name]=[method_name]
  return classes_and_methods, associations

In [ ]:
#@title ***write_class_diagram(classes_and_methods):*** Writes the puml file using the classes dictionary created
def write_class_diagram(classes_and_methods, associations):
  plantuml = """@startuml classDiagram

  <style>
      classDiagram{
          class{
              .view{
                  BackgroundColor lightgreen
              }
          }
      }
  </style>
  hide <<view>> stereotype
  """

  for class_name in classes_and_methods:
    plantuml += _break+"class "+class_name+"{"
    for method in classes_and_methods[class_name]:
      plantuml += _break+_tab+method
    plantuml += _break+"}"+_break
  plantuml += _break
  for association in associations:#                                                                             ASSOCIATION TEST *****
    for class_name in associations[association]:
      plantuml += _break+association+" -- "+class_name+_break
  plantuml += _break+"@enduml"
  return plantuml

#Main method

In [ ]:
#@title ***get_pumls(xml_file_folder)***
def get_pumls(xml_file_folder):
  pumls = []
  i=0
  for xml_file in get_xml_files(xml_file_folder):
    print("xml_file: ", xml_file)
    root_xml = ET.parse(xml_file).getroot()
    print("root_xml: ", root_xml)
    classes_and_methods, associations = get_classes_and_methods(root_xml)
    print("classes_and_methods: ", classes_and_methods)
    puml = write_class_diagram(classes_and_methods, associations)
    print("\n")
    if i==0:
      puml_name = 'class_diagram.puml'
    else:
      puml_name = 'class_diagram('+ str(i) +').puml'
    i+=1
    with open(puml_name, 'w') as file:
      file.write(puml)
    pumls.append(puml)
  return pumls

#Writing plantumls

In [ ]:
#@title Folder where the **xml files** of the phrases was uploaded
xml_file_folder = '/content'

In [ ]:
#@title Genrerate and print the pumls
i = 0
for puml in get_pumls(xml_file_folder):
  #print("Puml "+str(i)+":")
  i+=1
  print(puml)
  print("\n\n\n")

xml_file:  predicted_phrases_types (1).xml
root_xml:  <Element 'root' at 0x787f0e178270>
classes_and_methods:  {'V_Autenticação  <<view>>': ['Enviar_Acesso()'], 'C_Autenticação  <<controller>>': ['Enviar_Acesso()'], 'Acesso': ['Validar_Acesso()'], 'Produto': ['Nome', 'Validar_Produto()', 'Registrar_Produto()', 'Exibir_Produto()', 'Atualizar_Estoque()'], 'V_Cadastro_de_Produto  <<view>>': ['Enviar_Produto(Nome)'], 'C_Cadastro_de_Produto  <<controller>>': ['Enviar_Produto(Nome)'], 'Fabricante': ['Nome', 'Validar_Fabricante()', 'Registrar_Fabricante()'], 'V_Cadastro_do_Fabricante  <<view>>': ['Enviar_Fabricante(Nome)'], 'C_Cadastro_do_Fabricante  <<controller>>': ['Enviar_Fabricante(Nome)'], 'Grupo': ['Nome', 'Validar_Grupo()', 'Registrar_Grupo()', 'Validar_Grupo()'], 'V_Cadastro_de_Grupo  <<view>>': ['Enviar_Grupo(Nome)'], 'C_Cadastro_de_Grupo  <<controller>>': ['Enviar_Grupo(Nome)'], 'Subgrupo': ['Nome', 'Validar_Subgrupo()', 'Registrar_Subgrupo()'], 'V_Cadastro_de_Subrupo  <<view>>': [